In [38]:
import sys
import gc
import importlib
import datetime
import numpy as np
import netCDF4
import platform
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cartopy.crs as ccrs
import imageio
from tqdm import tqdm

# Define path
if platform.system() == 'Darwin':  # macOS
    base_FP = '/Users/subinkim/data'
    cpuserver_data_FP = '/Users/subinkim/cpuserver_data'
elif platform.system() == 'Linux':  # Linux systems (Workstation or CPU Server)
    base_FP = '/home/subin/data'
    cpuserver_data_FP = '/home/subin/cpuserver_data'
    if not os.path.exists(cpuserver_data_FP):
        cpuserver_data_FP = '/data'

print(base_FP)
print(cpuserver_data_FP)
sys.path.append(base_FP + '/python_modules')

import HydroAI.Data as hData
import HydroAI.Grid as hGrid
import HydroAI.TC_like as hTCL
import HydroAI.Plot as hPlot
#import HydroAI.Shallow_ML as hSML

importlib.reload(hData);
importlib.reload(hGrid);
importlib.reload(hTCL);
importlib.reload(hPlot);
#importlib.reload(hSML);

import warnings
warnings.filterwarnings("ignore")

/home/subin/data
/home/subin/cpuserver_data


#### 1. Read data

In [16]:
target_year = 2023

CYGNSS_data_path = f'{cpuserver_data_FP}/extracted_nc/CYGNSS/UCAR_CU_L3_SM_v1.0/CYGNSS_UCAR_{target_year}_R_Noah_025.nc'
with netCDF4.Dataset(CYGNSS_data_path) as nc:
    print(nc.variables.keys())
    nc_lon_CYGNSS = nc.variables['longitude'][:].data
    nc_lat_CYGNSS = nc.variables['latitude'][:].data
    SM_CYGNSS = nc.variables['SM_subdaily'][:].data
    print(np.shape(SM_CYGNSS))

SMAP_Sentinel1_data_path = f'{cpuserver_data_FP}/extracted_nc/SMAP_Sentinel1_L2_{target_year}_6_8_R_Noah_025.nc'
with netCDF4.Dataset(SMAP_Sentinel1_data_path) as nc:
    print(nc.variables.keys())
    nc_lon_SMAP_Sentinel1 = nc.variables['longitude'][:].data
    nc_lat_SMAP_Sentinel1 = nc.variables['latitude'][:].data
    SM_SMAP_Sentinel1 = nc.variables['soil_moisture_3km'][:].data
    print(np.shape(SM_SMAP_Sentinel1))

dict_keys(['latitude', 'longitude', 'SM_subdaily'])
(306, 1197, 368)
dict_keys(['latitude', 'longitude', 'soil_moisture_3km'])
(720, 1440, 92)


In [17]:
# Read ASCAT am & pm data
ASCAT_am_data_path = f'{cpuserver_data_FP}/extracted_nc/ASCAT_TUW/ASCAT_TUW_{target_year}_am_R_Noah_025.nc'
with netCDF4.Dataset(ASCAT_am_data_path) as nc:
    print(nc.variables.keys())
    nc_lon_ASCAT = nc.variables['longitude'][:].data
    nc_lat_ASCAT = nc.variables['latitude'][:].data
    SM_ASCAT_am = nc.variables['Resampled_ASCAT_SM_gldas'][:].data

ASCAT_pm_data_path = f'{cpuserver_data_FP}/extracted_nc/ASCAT_TUW/ASCAT_TUW_{target_year}_pm_R_Noah_025.nc'
with netCDF4.Dataset(ASCAT_pm_data_path) as nc:
    SM_ASCAT_pm = nc.variables['Resampled_ASCAT_SM_gldas'][:].data

# Only June to August
dates = np.array([datetime.date(target_year, 1, 1) + datetime.timedelta(days=i) for i in range(SM_ASCAT_am.shape[2])])
mask = np.isin([d.month for d in dates], [6, 7, 8])
SM_ASCAT_am = SM_ASCAT_am[:, :, mask]
SM_ASCAT_pm = SM_ASCAT_pm[:, :, mask]
print(np.shape(SM_ASCAT_am))
print(np.shape(SM_ASCAT_pm))

SM_ASCAT = np.full((SM_ASCAT_am.shape[0], SM_ASCAT_am.shape[1], 2*SM_ASCAT_am.shape[2]), np.nan)
SM_ASCAT[:, :, ::2] = SM_ASCAT_am
SM_ASCAT[:, :, 1::2] = SM_ASCAT_pm
SM_ASCAT = SM_ASCAT / 100 # Convert to m^3/m^3
print(np.shape(SM_ASCAT))

# Remove memory
del SM_ASCAT_am, SM_ASCAT_pm
gc.collect()

dict_keys(['latitude', 'longitude', 'Resampled_ASCAT_SM_gldas'])
(720, 1440, 92)
(720, 1440, 92)
(720, 1440, 184)


143359

In [18]:
# Read SMAP am & pm data
SMAP_am_data_path = f'{cpuserver_data_FP}/extracted_nc/SPL3SMP/SPL3SMP.008_{target_year}_am_R_Noah_025.nc'
with netCDF4.Dataset(SMAP_am_data_path) as nc:
    print(nc.variables.keys())
    nc_lon_SMAP = nc.variables['longitude'][:].data
    nc_lat_SMAP = nc.variables['latitude'][:].data
    SM_SMAP_am = nc.variables['Resampled_SMAP_SM'][:].data
    SM_SMAP_am_QC = nc.variables['Resampled_SMAP_SM_QC'][:].data

SMAP_pm_data_path = f'{cpuserver_data_FP}/extracted_nc/SPL3SMP/SPL3SMP.008_{target_year}_pm_R_Noah_025.nc'
with netCDF4.Dataset(SMAP_pm_data_path) as nc:
    SM_SMAP_pm = nc.variables['Resampled_SMAP_SM'][:].data
    SM_SMAP_pm_QC = nc.variables['Resampled_SMAP_SM_QC'][:].data

# Only June to August
SM_SMAP_am = SM_SMAP_am[:, :, mask]
SM_SMAP_pm = SM_SMAP_pm[:, :, mask]
SM_SMAP_am_QC = SM_SMAP_am_QC[:, :, mask]
SM_SMAP_pm_QC = SM_SMAP_pm_QC[:, :, mask]
print(np.shape(SM_SMAP_am), np.shape(SM_SMAP_pm), np.shape(SM_SMAP_am_QC), np.shape(SM_SMAP_pm_QC))

# Combine am & pm data
SM_SMAP = np.full((SM_SMAP_am.shape[0], SM_SMAP_am.shape[1], 2*SM_SMAP_am.shape[2]), np.nan)
SM_SMAP_QC = np.full((SM_SMAP_am.shape[0], SM_SMAP_am.shape[1], 2*SM_SMAP_am.shape[2]), np.nan)
SM_SMAP[:, :, ::2] = SM_SMAP_am
SM_SMAP[:, :, 1::2] = SM_SMAP_pm
SM_SMAP_QC[:, :, ::2] = SM_SMAP_am_QC
SM_SMAP_QC[:, :, 1::2] = SM_SMAP_pm_QC
print(np.shape(SM_SMAP), np.shape(SM_SMAP_QC))

# Remove memory
del SM_SMAP_am, SM_SMAP_pm, SM_SMAP_am_QC, SM_SMAP_pm_QC
gc.collect()

dict_keys(['latitude', 'longitude', 'Resampled_albedo', 'Resampled_albedo_dca', 'Resampled_albedo_scah', 'Resampled_albedo_scav', 'Resampled_boresight_incidence', 'Resampled_bulk_density', 'Resampled_clay_fraction', 'Resampled_freeze_thaw_fraction', 'Resampled_grid_surface_status', 'Resampled_radar_water_body_fraction', 'Resampled_retrieval_qual_flag_dca', 'Resampled_retrieval_qual_flag_scah', 'Resampled_retrieval_qual_flag_scav', 'Resampled_roughness_coefficient', 'Resampled_roughness_coefficient_dca', 'Resampled_roughness_coefficient_scah', 'Resampled_roughness_coefficient_scav', 'Resampled_soil_moisture_dca', 'Resampled_soil_moisture_error', 'Resampled_soil_moisture_scah', 'Resampled_soil_moisture_scav', 'Resampled_static_water_body_fraction', 'Resampled_surface_flag', 'Resampled_surface_temperature', 'Resampled_surface_water_fraction_mb_h', 'Resampled_surface_water_fraction_mb_v', 'Resampled_tb_3_corrected', 'Resampled_tb_4_corrected', 'Resampled_tb_h_corrected', 'Resampled_tb_h_un

60

#### 2. Generate MP4 files

In [29]:
# Check data range
print(np.nanmin(SM_SMAP_Sentinel1), np.nanmax(SM_SMAP_Sentinel1))
print(np.nanmin(SM_CYGNSS), np.nanmax(SM_CYGNSS))
print(np.nanmin(SM_SMAP), np.nanmax(SM_SMAP), np.nanmin(SM_SMAP_QC), np.nanmax(SM_SMAP_QC))
print(np.nanmin(SM_ASCAT), np.nanmax(SM_ASCAT))

0.02 0.75
0.0 0.6499589
0.019999999552965164 0.49999943375587463 0.019999999552965164 0.49999943375587463
0.0 0.96


In [ ]:
# Daily
hPlot.generate_MP4_rotating_globe(
    nc_lon_SMAP_Sentinel1, nc_lat_SMAP_Sentinel1, data_3d=SM_SMAP_Sentinel1,
    vmin=0.1, vmax=0.5,
    output_mp4=f'{cpuserver_data_FP}/extracted_mp4/SMAP_Sentinel1_L2_{target_year}_6_8_R_Noah_025.mp4', 
    plot_title='SMAP/Sentinel-1 SM (Daily)',
    cbar_title='SM (m$^3$/m$^3$)',
    frame_duration=0.2,                
    lons_center=np.tile(np.arange(-180, 180, 10), 2),
    init_time_idx=0, time_step=1,
    time_length_per_rotation=1,
    cmap='jet_r')

# Subdaily (6-hourly)
hPlot.generate_MP4_rotating_globe(
    nc_lon_CYGNSS, nc_lat_CYGNSS, data_3d=SM_CYGNSS,
    vmin=0.1, vmax=0.5,
    output_mp4=f'{cpuserver_data_FP}/extracted_mp4/CYGNSS_UCAR_{target_year}_6_8_R_Noah_025.mp4', 
    plot_title='CYGNSS SM (6-hourly)',
    cbar_title='SM (m$^3$/m$^3$)',
    frame_duration=0.05, # 0.05 for 6-hourly, 0.2 for daily               
    lons_center=np.tile(np.arange(-180, 180, 10), 2),
    init_time_idx=0, time_step=1,
    time_length_per_rotation=4, # 4 for 6-hourly, 1 for daily
    cmap='jet_r')

# Subdaily (12-hourly)
hPlot.generate_MP4_rotating_globe(
    nc_lon_ASCAT, nc_lat_ASCAT, data_3d=SM_ASCAT,
    vmin=0.1, vmax=0.5,
    output_mp4=f'{cpuserver_data_FP}/extracted_mp4/ASCAT_TUW_{target_year}_6_8_R_Noah_025.mp4', 
    plot_title='ASCAT SM (12-hourly)',
    cbar_title='SM (m$^3$/m$^3$)',
    frame_duration=0.1, # 0.1 for 12-hourly, 0.2 for daily               
    lons_center=np.tile(np.arange(-180, 180, 10), 2),
    init_time_idx=0, time_step=1,
    time_length_per_rotation=2, # 2 for 12-hourly, 1 for daily
    cmap='jet_r')

hPlot.generate_MP4_rotating_globe(
    nc_lon_SMAP, nc_lat_SMAP, data_3d=SM_SMAP,
    vmin=0.1, vmax=0.5,
    output_mp4=f'{cpuserver_data_FP}/extracted_mp4/SMAP_{target_year}_6_8_R_Noah_025.mp4', 
    plot_title='SMAP SM (12-hourly)',
    cbar_title='SM (m$^3$/m$^3$)',
    frame_duration=0.1, # 0.1 for 12-hourly, 0.2 for daily               
    lons_center=np.tile(np.arange(-180, 180, 10), 2),
    init_time_idx=0, time_step=1,
    time_length_per_rotation=2, # 2 for 12-hourly, 1 for daily
    cmap='jet_r')

/home/subin/.env


Rendering frames (MP4): 100%|██████████| 72/72 [03:42<00:00,  3.09s/it]


MP4 saved to /home/subin/cpuserver_data/extracted_mp4/SMAP_Sentinel1_L2_2023_6_8_R_Noah_025.mp4 (fps=5, codec=libx264)


Rendering frames (MP4): 100%|██████████| 72/72 [09:46<00:00,  8.15s/it]


MP4 saved to /home/subin/cpuserver_data/extracted_mp4/CYGNSS_UCAR_2023_6_8_R_Noah_025.mp4 (fps=20, codec=libx264)


Rendering frames (MP4): 100%|██████████| 72/72 [06:38<00:00,  5.54s/it]


MP4 saved to /home/subin/cpuserver_data/extracted_mp4/ASCAT_TUW_2023_6_8_R_Noah_025.mp4 (fps=10, codec=libx264)


Rendering frames (MP4): 100%|██████████| 72/72 [01:49<00:00,  1.52s/it]


MP4 saved to /home/subin/cpuserver_data/extracted_mp4/SMAP_2023_6_8_R_Noah_025.mp4 (fps=10, codec=libx264)
